# Missing Values

Brendan Reed  
12/17/2025

## Description
- Many branches have missing values where sizes are not recorded properly  
- also check where # packs is not recorded

## Imports

### Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

### Set Display Options

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

### Import Data

In [3]:
df = pd.read_csv("data/diapers.csv")

df.head()

,Timestamp,BRANCH,ZIP CODE,SIZE,# PACKS,SIZE.1,# PACKS.1,SIZE.2,# PACKS.2,TOTAL,DATE,MONTH,YEAR,DAY,WEEKDAY
0,9/7/2021 0:00:00,GG,63011,1,3.0,NaN,NaN,NaN,NaN,3,September 2021,September,2021,7,Tue
1,9/7/2021 0:00:00,GG,63011,3T,2.0,NaN,NaN,NaN,NaN,2,September 2021,September,2021,7,Tue
2,9/14/2021 0:00:00,GG,63118,5,2.0,NaN,NaN,NaN,NaN,2,September 2021,September,2021,14,Tue
3,9/14/2021 0:00:00,GG,63118,6,2.0,NaN,NaN,NaN,NaN,2,September 2021,September,2021,14,Tue
4,9/14/2021 0:00:00,GG,63125,5,2.0,NaN,NaN,NaN,NaN,2,September 2021,September,2021,14,Tue


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59425 entries, 0 to 59424
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Timestamp  50120 non-null  object 
 1   BRANCH     50117 non-null  object 
 2   ZIP CODE   50113 non-null  object 
 3   SIZE       49967 non-null  object 
 4   # PACKS    50090 non-null  float64
 5   SIZE.1     7687 non-null   object 
 6   # PACKS.1  7595 non-null   float64
 7   SIZE.2     1648 non-null   object 
 8   # PACKS.2  2491 non-null   float64
 9   TOTAL      59425 non-null  int64  
 10  DATE       59425 non-null  object 
 11  MONTH      50120 non-null  object 
 12  YEAR       50120 non-null  object 
 13  DAY        50120 non-null  object 
 14  WEEKDAY    50120 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 6.8+ MB


## Cleaning Data

Choose only desired columns

In [5]:
df = df[[
    'Timestamp', 'BRANCH', 'ZIP CODE', 'SIZE', '# PACKS', 'SIZE.1',
    '# PACKS.1', 'SIZE.2', '# PACKS.2', 'TOTAL'
]]

Make all column names lower case

In [6]:
df.columns = [item.lower() for item in list(df.columns)]

Create a row for each pair of sizes and # packs

In [7]:
size_cols = ["size", "size.1", "size.2"]
pack_cols = ["# packs", "# packs.1", "# packs.2"]

pieces = []

for i in range(3):  # force 3 iterations
    temp = df[["timestamp", "branch", "zip code"]].copy()
    temp["size"] = df[size_cols[i]]
    temp["# packs"] = df[pack_cols[i]]
    pieces.append(temp)

df = pd.concat(pieces, ignore_index=True)


Drop rows where both size and # packs is NA

In [8]:
df = df.dropna(subset=["size", "# packs"], how="all")

Preview data

Check for missing values

In [9]:
pd.DataFrame({
    "NaN Count": df.isna().sum(),
    "Total Rows": len(df),
    "Percent NaN": df.isna().mean() * 100
})

,NaN Count,Total Rows,Percent NaN
timestamp,1,60382,0.001656
branch,6,60382,0.009937
zip code,9,60382,0.014905
size,1080,60382,1.788613
# packs,206,60382,0.341161


Size is missing for 444 rows, which we will need to address. For now, I will leave missing values and see how prevalent they are across different months.

See unique sizes

In [10]:
df["size"].unique()

array(['1', '3T', '5', '6', '3', 'N', '4T', '4', '2', '4t', '2T', '3t',
       '2t', 'n', '7', 'adult', 'P', '0', nan], dtype=object)

Some sizes have duplicates or are not valid.  
Fixes: 
- 'n' and 'N' need to be combined
- 2t and 2T need to be combined for pullups (same with 3t/3T and 4t/4T)

In [11]:
# cast all sizes to strings
df['size'] = df['size'].astype(str)

# make all letters in df["size"] upper case
df["size"] = [x.upper() for x in df["size"]]

# Replace real NaN in size with a category name
df['size_cat'] = df['size'].astype('object').fillna('NAN')

Create date and month columns

In [12]:
# convert timestamp column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], errors="coerce")

# create month column
df['month'] = df['timestamp'].dt.to_period('M')

# create date column
df['date'] = pd.to_datetime(df['timestamp']).dt.date

# remove timestamp column
# df = df.drop('timestamp', axis=1)


## Missing Values

### Missing Size

In [13]:
df_nan = df[df["size"] == "NAN"]
df_nan_filtered = df_nan[df_nan["month"].between("2025-01", "2025-11")]
df_nan_filtered.head()


,timestamp,branch,zip code,size,# packs,size_cat,month,date
48837,2025-11-30 15:11:29,FV,63033,NAN,1.0,NAN,2025-11,2025-11-30
48853,2025-11-26 13:03:30,NB,63115,NAN,1.0,NAN,2025-11,2025-11-26
48861,2025-11-26 10:59:16,FV,63033,NAN,1.0,NAN,2025-11,2025-11-26
48871,2025-11-25 17:14:23,FV,63136,NAN,1.0,NAN,2025-11,2025-11-25
48872,2025-11-25 17:06:33,NB,63118,NAN,1.0,NAN,2025-11,2025-11-25


In [14]:
pivot = pd.pivot_table(df_nan_filtered, index='branch', values='# packs', aggfunc='sum',columns='month')
pivot['total'] = pivot.sum(axis=1)
pivot['total'] = [math.ceil(x) for x in pivot['total']]
pivot['total'].to_frame()

,total
branch,
FV,52
GG,34
LC,35
NB,407
PV,39
WR,309


### Missing # Packs

In [15]:
df_packs_nan = df[df["# packs"].isna()]
df_packs_nan_filtered = df_packs_nan[df_packs_nan["month"].between("2025-01", "2025-11")]
df_packs_nan_filtered.head()

,timestamp,branch,zip code,size,# packs,size_cat,month,date
48842,2025-11-26 16:03:23,FV,63031,3,NaN,3,2025-11,2025-11-26
48851,2025-11-26 13:32:34,PV,63129,6,NaN,6,2025-11,2025-11-26
48852,2025-11-26 13:31:47,PV,63129,4,NaN,4,2025-11,2025-11-26
48957,2025-11-20 19:09:37,FV,63114,3,NaN,3,2025-11,2025-11-20
49026,2025-11-19 09:01:36,LC,63121,6,NaN,6,2025-11,2025-11-19


In [16]:
pivot = pd.pivot_table(df_packs_nan_filtered, index='branch', values='size', aggfunc='count',columns='month')
pivot['total'] = pivot.sum(axis=1)
pivot

month,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,total
branch,,,,,,,,,,,,
FV,1.0,1.0,1.0,2.0,1.0,1.0,NaN,5.0,NaN,7.0,8.0,27.0
GG,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,3.0
LC,1.0,1.0,1.0,NaN,NaN,1.0,3.0,NaN,9.0,8.0,5.0,29.0
NB,1.0,NaN,2.0,NaN,NaN,3.0,NaN,NaN,NaN,1.0,NaN,7.0
PV,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,6.0,8.0
WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,4.0,6.0
